In [ ]:
input_folder = r"i:\SpineUs\2024_Segmentation\02_PreparedArrays_2pre"

num_sample_frames = 3

In [ ]:
# Check files in the input_folder. Filenames follow this pattern: "PatientID_Orientation_Type.npy"
# PatientID is a string of numerical digits of arbitrary length
# ScanID is a string, e.g., "Ax", "Sa", "SaL", "SaT", "Sa-LU", etc.
# Type is a string that can only have the following values: "indices", "segmentation", "transform", or "ultrasound"

# Count how many unique PatientID ScanID pairs there are in the input_folder that have all four types of files

import os
import re
import numpy as np

# Get all files in the input_folder
files = os.listdir(input_folder)

# Print how many files are in the input_folder in total
print(f"Total number of files: {len(files)}")

# Create a dictionary to store the files
file_dict = {}

# Loop through all files

for file in files:
    # Extract the PatientID, ScanID, and Type from the filename. Extension can either be npy or npz.
    match = re.match(r"(\d+)_([A-Za-z-]+)_(indices|segmentation|transform|ultrasound)\.(npy|npz)", file)
    if match:
        PatientID = match.group(1)
        ScanID = match.group(2)
        Type = match.group(3)
        
        # If the PatientID is not in the dictionary, add it
        if PatientID not in file_dict:
            file_dict[PatientID] = {}
        
        # If the Orientation is not in the dictionary, add it
        if ScanID not in file_dict[PatientID]:
            file_dict[PatientID][ScanID] = set()
        
        # Add the Type to the set
        file_dict[PatientID][ScanID].add(Type)

# Count how many unique PatientID ScanID pairs have all four types of files
count = 0
for PatientID in file_dict:
    for ScanID in file_dict[PatientID]:
        if len(file_dict[PatientID][ScanID]) == 4:
            count += 1

# Remove all ScanIDs that do not have all four types of files.
# Remove all PatientIDs that do not have any ScanIDs left.
count_scans_removed = 0
for PatientID in list(file_dict.keys()):
    for ScanID in list(file_dict[PatientID].keys()):
        if len(file_dict[PatientID][ScanID]) != 4:
            del file_dict[PatientID][ScanID]
            count_scans_removed += 1
    if len(file_dict[PatientID]) == 0:
        del file_dict[PatientID]
print(f"Number of scans removed for missing file types: {count_scans_removed}")

print(f"Number of PatientID_Scan pairs with all types of files: {count}")

In [ ]:
# Read the indices, segmentation, transform, and ultrasound files for one of the PatientID_Scan pairs
PatientID = list(file_dict.keys())[0]
ScanID = list(file_dict[PatientID].keys())[0]
indices = np.load(os.path.join(input_folder, f"{PatientID}_{ScanID}_indices.npz"))
if isinstance(indices, np.lib.npyio.NpzFile):
    indices = indices[indices.files[0]]
segmentation = np.load(os.path.join(input_folder, f"{PatientID}_{ScanID}_segmentation.npy"))
transform = np.load(os.path.join(input_folder, f"{PatientID}_{ScanID}_transform.npy"))
ultrasound = np.load(os.path.join(input_folder, f"{PatientID}_{ScanID}_ultrasound.npy"))

print(f"PatientID:          {PatientID}")
print(f"ScanID:             {ScanID}")
print()
# Print the shapes of the arrays
print(f"indices shape:      {indices.shape}")
print(f"transform shape:    {transform.shape}")
print()
print(f"segmentation shape: {segmentation.shape}")
print(f"segmentation dtype: {segmentation.dtype}")
print(f"segmentation min:   {segmentation.min()}")
print(f"segmentation max:   {segmentation.max()}")
print()
print(f"ultrasound shape:   {ultrasound.shape}")
print(f"ultrasound dtype:   {ultrasound.dtype}")
print(f"ultrasound min:     {ultrasound.min()}")
print(f"ultrasound max:     {ultrasound.max()}")


In [ ]:
# Ultrasound, segmentation, and transform data dimensions start with frame count.
# Ultrasound and segmentation data dimeensions are: [frame count, height, width, channels]

# Pick a num_sample_frames random frame numbers between 0 and the frame count
frame_indices = np.random.choice(indices.shape[0], num_sample_frames, replace=False)

# Show the ultrasound frames and corresponding segmentations side by side
import matplotlib.pyplot as plt

fig, axs = plt.subplots(num_sample_frames, 3, figsize=(12, num_sample_frames * 4))
for i, frame_index in enumerate(frame_indices):
    axs[i, 0].imshow(ultrasound[frame_index, ..., 0], cmap="gray")
    axs[i, 0].set_title(f"Ultrasound Frame {frame_index}")
    axs[i, 0].axis("off")
    
    axs[i, 1].imshow(segmentation[frame_index, ..., 0], cmap="gray")
    axs[i, 1].set_title(f"Segmentation Frame {frame_index}")
    axs[i, 1].axis("off")
    
    combined_image = np.stack([ultrasound[frame_index, ..., 0]]*3, axis=-1)  # Copy the ultrasound frame three times
    combined_image[..., 2] = np.maximum(combined_image[..., 2], segmentation[frame_index, ..., 0]*255)  # Overlay segmentation
    axs[i, 2].imshow(combined_image)
    axs[i, 2].set_title(f"Combined Frame {frame_index}")
    axs[i, 2].axis("off")
plt.tight_layout()
plt.show()